In [19]:
import os
import cv2
from ultralytics import YOLO
import numpy as np

In [20]:
# Load a pre-trained YOLO model for segmentation
model = YOLO('yolo11m-seg.pt')  # Consider a larger model for better accuracy
model.yaml = {'nc': 2} 

In [21]:

# Train the model with the updated configuration and hyperparameters
model.train(
    data='config.yaml',  # Path to your configuration file
    epochs=20,           # Set to at least 10 for initial testing
    imgsz=640,           # Image size
    batch=4,            # Batch size
    lr0=0.01,
    momentum=0.937,      
    weight_decay=0.0005,
    warmup_epochs=3,    
    save_period=5,            
)


New https://pypi.org/project/ultralytics/8.3.38 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.32  Python-3.12.0 torch-2.5.1+cpu CPU (11th Gen Intel Core(TM) i7-11370H 3.30GHz)
engine\trainer: task=segment, mode=train, model=yolo11m-seg.pt, data=config.yaml, epochs=20, time=None, patience=100, batch=4, imgsz=640, save=True, save_period=5, cache=False, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fra

train: Scanning C:\Users\baner\Desktop\BTP\btpf2\BTPF2-20241117T165856Z-001\BTPF2\labels\train.cache... 30 images, 0 ba
val: Scanning C:\Users\baner\Desktop\BTP\btpf2\BTPF2-20241117T165856Z-001\BTPF2\labels\val.cache... 9 images, 0 backgro

Plotting labels to runs\segment\train7\labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 115 weight(decay=0.0), 126 weight(decay=0.0005), 125 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\segment\train7
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/8 [00:20<?, ?it/s]

KeyboardInterrupt



In [26]:
# Define paths
model_path = r"runs\segment\train\weights\last.pt"#Relative path
input_folder = r"C:\Users\baner\Desktop\BTP\btpf2\BTPF2-20241117T165856Z-001\BTPF2\captures\val"#Absolute path for i/p folder
output_folder = r"C:\Users\baner\Desktop\BTP\btpf2\BTPF2-20241117T165856Z-001\BTPF2\20ep\op"#Absolute path of o/p folder

In [27]:
# Ensure output directory exists
os.makedirs(output_folder, exist_ok=True)

# Load the pre-trained YOLO model
model = YOLO(model_path)

In [28]:
model

YOLO(
  (model): SegmentationModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, tra

In [25]:
# # Process all images in the input folder
# for image_file in os.listdir(input_folder):
#     image_path = os.path.join(input_folder, image_file)
    
#     if not image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
#         continue

#     # Initialize variables for each image
#     dimensions = []  # Store width and contours for all detected objects
#     calculated_length = None
#     calculated_width = None

#     # Load the image
#     img = cv2.imread(image_path)
#     H, W, _ = img.shape

#     # Run the model on the image
#     results = model(img)

#     # First pass: Collect dimensions for all objects
#     for result in results:
#         for mask in result.masks.data:
#             mask = mask.numpy() * 255
#             mask = cv2.resize(mask, (W, H), interpolation=cv2.INTER_NEAREST)
#             _, thresh = cv2.threshold(mask.astype(np.uint8), 127, 255, cv2.THRESH_BINARY)
#             contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#             for cnt in contours:
#                 if cv2.contourArea(cnt) > 200:  # Filter small contours
#                     x, y, w, h = cv2.boundingRect(cnt)
#                     dimensions.append((w, h, cnt))  # Store width, height, and contour

#     # Sort objects by width
#     dimensions.sort(key=lambda x: x[0])  # Sort by width (ascending)

#     # Assign the smallest width as the rectangle, largest as the drop
#     if len(dimensions) >= 2:  # Ensure at least two objects exist
#         wrect, hrect, rect_cnt = dimensions[0]
#         wdrop, ldrop, drop_cnt = dimensions[-1]

#         # Calculate drop's dimensions using rectangle width
#         if wrect > 0:
#             calculated_length = ((ldrop / 2wrect) * 2, 5)
#             calculated_width = round((wdrop / wrect) * 2, 5)

#         # Draw rectangle bounding box
#         x, y, w, h = cv2.boundingRect(rect_cnt)
#         cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
#         cv2.putText(img, "Rectangle", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

#         # Draw drop bounding box
#         x, y, w, h = cv2.boundingRect(drop_cnt)
#         cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
#         cv2.putText(img, "Drop", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

#         # Annotate calculated results
#         if calculated_length is not None and calculated_width is not None:
#             cv2.putText(img, f"Length: {calculated_length} mm", (x, y - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
#             cv2.putText(img, f"Width: {calculated_width} mm", (x, y - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

#     # Save the final image with bounding boxes and labels
#     output_image_path = os.path.join(output_folder, image_file)
#     cv2.imwrite(output_image_path, img)

#     # Print the results for the current image
#     print(f"Processed {image_file}:")
#     if len(dimensions) >= 2:
#         print(f"  Width of rectangle: {wrect}px")
#         print(f"  Width of drop: {wdrop}px")
#         print(f"  Length of drop: {ldrop}px")
#         print(f"  Calculated width: {calculated_width} mm")
#         print(f"  Calculated length: {calculated_length} mm")
#     else:
#         print("  Not enough objects to classify (require at least 2).")
#     print(f"  Output saved to: {output_image_path}")

In [24]:
# import csv


# # Define the CSV output file
# csv_output_path = os.path.join(output_folder, "results.csv")

# # Initialize the CSV file with headers if it doesn't already exist
# if not os.path.exists(csv_output_path):
#     with open(csv_output_path, mode='w', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(["Image Name", "Calculated Width (mm)", "Calculated Length (mm)"])

# # Process all images in the input folder
# for image_file in os.listdir(input_folder):
#     image_path = os.path.join(input_folder, image_file)
    
#     if not image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
#         continue

#     # Initialize variables for each image
#     dimensions = []  # Store width and contours for all detected objects
#     calculated_length = None
#     calculated_width = None

#     # Load the image
#     img = cv2.imread(image_path)
#     H, W, _ = img.shape

#     # Run the model on the image
#     results = model(img)

#     # First pass: Collect dimensions for all objects
#     for result in results:
#         for mask in result.masks.data:
#             mask = mask.numpy() * 255
#             mask = cv2.resize(mask, (W, H), interpolation=cv2.INTER_NEAREST)
#             _, thresh = cv2.threshold(mask.astype(np.uint8), 127, 255, cv2.THRESH_BINARY)
#             contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#             for cnt in contours:
#                 if cv2.contourArea(cnt) > 200:  # Filter small contours
#                     x, y, w, h = cv2.boundingRect(cnt)
#                     dimensions.append((w, h, cnt))  # Store width, height, and contour

#     # Sort objects by width
#     dimensions.sort(key=lambda x: x[0])  # Sort by width (ascending)

#     # Assign the smallest width as the rectangle, largest as the drop
#     if len(dimensions) >= 2:  # Ensure at least two objects exist
#         wrect, hrect, rect_cnt = dimensions[0]
#         wdrop, ldrop, drop_cnt = dimensions[-1]

#         # Calculate drop's dimensions using rectangle width
#         if wrect > 0:
#             calculated_length = round((ldrop / wrect*2) * 2, 5)  # Calculate length
#             calculated_width = round((wdrop / wrect) * 2, 5)   # Calculate width

#         # Draw rectangle bounding box
#         x, y, w, h = cv2.boundingRect(rect_cnt)
#         cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
#         cv2.putText(img, "Rectangle", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

#         # Draw drop bounding box
#         x, y, w, h = cv2.boundingRect(drop_cnt)
#         cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
#         cv2.putText(img, "Drop", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

#         # Annotate calculated results
#         if calculated_length is not None and calculated_width is not None:
#             cv2.putText(img, f"Length: {calculated_length} mm", (x, y - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
#             cv2.putText(img, f"Width: {calculated_width} mm", (x, y - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

#     # Save the final image with bounding boxes and labels
#     output_image_path = os.path.join(output_folder, image_file)
#     cv2.imwrite(output_image_path, img)

#     # Write results to the CSV file
#     with open(csv_output_path, mode='a', newline='') as file:
#         writer = csv.writer(file)
#         if len(dimensions) >= 2:
#             writer.writerow([image_file, calculated_width, calculated_length])
#         else:
#             writer.writerow([image_file, "N/A", "N/A"])

#     # Print the results for the current image
#     print(f"Processed {image_file}:")
#     if len(dimensions) >= 2:
#         print(f"  Calculated width: {calculated_width} mm")
#         print(f"  Calculated length: {calculated_length} mm")
#     else:
#         print("  Not enough objects to classify (require at least 2).")
#     print(f"  Output saved to: {output_image_path}")


In [28]:
# # Process all images in the input folder
# for image_file in os.listdir(input_folder):
#     image_path = os.path.join(input_folder, image_file)
    
#     if not image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
#         continue

#     # Initialize variables for each image
#     dimensions = []  # Store width and contours for all detected objects
#     calculated_length = None
#     calculated_width = None
#     drop_width_at_D = None  # To store the width at distance D

#     # Load the image
#     img = cv2.imread(image_path)
#     H, W, _ = img.shape

#     # Run the model on the image
#     results = model(img)

#     # First pass: Collect dimensions for all objects
#     for result in results:
#         for mask in result.masks.data:
#             mask = mask.numpy() * 255
#             mask = cv2.resize(mask, (W, H), interpolation=cv2.INTER_NEAREST)
#             _, thresh = cv2.threshold(mask.astype(np.uint8), 127, 255, cv2.THRESH_BINARY)
#             contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#             for cnt in contours:
#                 if cv2.contourArea(cnt) > 200:  # Filter small contours
#                     x, y, w, h = cv2.boundingRect(cnt)
#                     dimensions.append((w, h, cnt, mask))  # Store width, height, contour, and mask

#     # Sort objects by width
#     dimensions.sort(key=lambda x: x[0])  # Sort by width (ascending)

#     # Assign the smallest width as the rectangle, largest as the drop
#     if len(dimensions) >= 2:  # Ensure at least two objects exist
#         wrect, hrect, rect_cnt, rect_mask = dimensions[0]
#         wdrop, ldrop, drop_cnt, drop_mask = dimensions[-1]

#         # Calculate drop's dimensions using rectangle width
#         if wrect > 0:
#             calculated_length = round((ldrop / wrect) * 2, 5)
#             calculated_width = round((wdrop / wrect) * 2, 5)

#         # Compute width at distance D from top of the bounding box around the drop
#         D = wdrop  # Set D as the width of the bounding box
#         row_at_D = int(D)  # Convert D to integer row index
#         top_y = cv2.boundingRect(drop_cnt)[1]  # Get top y-coordinate of the bounding box
#         row_y = top_y + row_at_D  # Calculate the y-coordinate for row D

#         # Ensure the row is within the mask boundaries
#         if 0 <= row_y < drop_mask.shape[0]:
#             row_pixels = drop_mask[row_y, :]  # Extract the row of the mask
#             cols = np.where(row_pixels > 0)[0]  # Find non-zero (mask) pixels
#             if len(cols) > 0:
#                 drop_width_at_D = cols[-1] - cols[0]  # Calculate the width
#                 calculated_d = round((drop_width_at_D / wrect) * 2, 5)

#         # Draw rectangle bounding box
#         x, y, w, h = cv2.boundingRect(rect_cnt)
#         cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
#         cv2.putText(img, "Rectangle", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

#         # Draw drop bounding box
#         x, y, w, h = cv2.boundingRect(drop_cnt)
#         cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
#         cv2.putText(img, "Drop", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

#         # Annotate calculated results
#         if calculated_length is not None and calculated_width is not None:
#             cv2.putText(img, f"Length: {calculated_length} mm", (x, y - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
#             cv2.putText(img, f"Width: {calculated_width} mm", (x, y - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
#         if drop_width_at_D is not None:
#             cv2.putText(img, f"Width at D: {calculated_d} px", (x, y - 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

#     # Save the final image with bounding boxes and labels
#     output_image_path = os.path.join(output_folder, image_file)
#     cv2.imwrite(output_image_path, img)

#     # Print the results for the current image
#     print(f"Processed {image_file}:")
#     if len(dimensions) >= 2:
#         print(f"  Width of rectangle: {wrect}px")
#         print(f"  Width of drop : {wdrop}px")
#         print(f"  Length of drop: {ldrop}px")
#         print(f"  Calculated width (D) : {calculated_width} mm")
#         print(f"  Calculated length: {calculated_length} mm")
#         print(f"  Drop width at D: {drop_width_at_D} px")
#         print(f"  d: {calculated_d} px")
#     else:
#         print("  Not enough objects to classify (require at least 2).")
#     print(f"  Output saved to: {output_image_path}")



0: 480x640 1 rectangle, 1 drop, 1916.3ms
Speed: 14.1ms preprocess, 1916.3ms inference, 17.2ms postprocess per image at shape (1, 3, 480, 640)
Processed 20240531_165251_NiyGB.png:
  Width of rectangle: 44px
  Width of drop : 108px
  Length of drop: 124px
  Calculated width (D) : 4.90909 mm
  Calculated length: 5.63636 mm
  Drop width at D: 71 px
  d: 3.22727 px
  Output saved to: C:\Users\baner\Desktop\BTP\btpf2\BTPF2-20241117T165856Z-001\BTPF2\20ep\op\20240531_165251_NiyGB.png

0: 480x640 1 rectangle, 1 drop, 1928.5ms
Speed: 9.4ms preprocess, 1928.5ms inference, 13.1ms postprocess per image at shape (1, 3, 480, 640)
Processed 20240531_165254_CACT9.png:
  Width of rectangle: 44px
  Width of drop : 108px
  Length of drop: 120px
  Calculated width (D) : 4.90909 mm
  Calculated length: 5.45455 mm
  Drop width at D: 68 px
  d: 3.09091 px
  Output saved to: C:\Users\baner\Desktop\BTP\btpf2\BTPF2-20241117T165856Z-001\BTPF2\20ep\op\20240531_165254_CACT9.png

0: 480x640 1 rectangle, 1 drop, 17

KeyboardInterrupt: 

In [29]:

import csv

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Path to the output CSV file
csv_file_path = os.path.join(output_folder, "results.csv")

# Open the CSV file for writing
with open(csv_file_path, mode="w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    # Write the header
    csv_writer.writerow(["Image Name", "Calculated Width (mm)", "Calculated D (mm)"])

    # Process all images in the input folder
    for image_file in os.listdir(input_folder):
        image_path = os.path.join(input_folder, image_file)

        if not image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            continue

        # Initialize variables for each image
        dimensions = []  # Store width and contours for all detected objects
        calculated_length = None
        calculated_width = None
        drop_width_at_D = None  # To store the width at distance D
        calculated_d = None  # To store calculated D in mm

        # Load the image
        img = cv2.imread(image_path)
        H, W, _ = img.shape

        # Run the model on the image
        results = model(img)

        # First pass: Collect dimensions for all objects
        for result in results:
            for mask in result.masks.data:
                mask = mask.numpy() * 255
                mask = cv2.resize(mask, (W, H), interpolation=cv2.INTER_NEAREST)
                _, thresh = cv2.threshold(mask.astype(np.uint8), 127, 255, cv2.THRESH_BINARY)
                contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                for cnt in contours:
                    if cv2.contourArea(cnt) > 200:  # Filter small contours
                        x, y, w, h = cv2.boundingRect(cnt)
                        dimensions.append((w, h, cnt, mask))  # Store width, height, contour, and mask

        # Sort objects by width
        dimensions.sort(key=lambda x: x[0])  # Sort by width (ascending)

        # Assign the smallest width as the rectangle, largest as the drop
        if len(dimensions) >= 2:  # Ensure at least two objects exist
            wrect, hrect, rect_cnt, rect_mask = dimensions[0]
            wdrop, ldrop, drop_cnt, drop_mask = dimensions[-1]

            # Calculate drop's dimensions using rectangle width
            if wrect > 0:
                calculated_length = round((ldrop / wrect) * 1.27, 5)
                calculated_width = round((wdrop / wrect) * 1.27, 5)

            # Compute width at distance D from top of the bounding box around the drop
            D = wdrop  # Set D as the width of the bounding box
            row_at_D = int(D)  # Convert D to integer row index
            top_y = cv2.boundingRect(drop_cnt)[1]  # Get top y-coordinate of the bounding box
            row_y = top_y + row_at_D  # Calculate the y-coordinate for row D

            # Ensure the row is within the mask boundaries
            if 0 <= row_y < drop_mask.shape[0]:
                row_pixels = drop_mask[row_y, :]  # Extract the row of the mask
                cols = np.where(row_pixels > 0)[0]  # Find non-zero (mask) pixels
                if len(cols) > 0:
                    drop_width_at_D = cols[-1] - cols[0]  # Calculate the width
                    calculated_d = round((drop_width_at_D / wrect) * 1.27, 5)

            # Draw rectangle bounding box
            x, y, w, h = cv2.boundingRect(rect_cnt)
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(img, "Rectangle", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

            # Draw drop bounding box
            x, y, w, h = cv2.boundingRect(drop_cnt)
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img, "Drop", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            # Annotate calculated results
            if calculated_length is not None and calculated_width is not None:
                cv2.putText(img, f"Length: {calculated_length} mm", (x, y - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(img, f"Width: {calculated_width} mm", (x, y - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            if drop_width_at_D is not None:
                cv2.putText(img, f"Width at D: {calculated_d} mm", (x, y - 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # Save the final image with bounding boxes and labels
        output_image_path = os.path.join(output_folder, image_file)
        cv2.imwrite(output_image_path, img)

        # Save results to the CSV
        csv_writer.writerow([image_file, calculated_width, calculated_d])

        # Print the results for the current image
        print(f"Processed {image_file}:")
        if len(dimensions) >= 2:
            print(f"  Calculated width: {calculated_width} mm")
            print(f"  Calculated D: {calculated_d} mm")
        else:
            print("  Not enough objects to classify (require at least 2).")
        print(f"  Output saved to: {output_image_path}")

print(f"Results saved to {csv_file_path}")



0: 480x640 1 rectangle, 1 drop, 1176.7ms
Speed: 9.0ms preprocess, 1176.7ms inference, 3.8ms postprocess per image at shape (1, 3, 480, 640)
Processed 20240531_165251_NiyGB.png:
  Calculated width: 3.11727 mm
  Calculated D: 2.04932 mm
  Output saved to: C:\Users\baner\Desktop\BTP\btpf2\BTPF2-20241117T165856Z-001\BTPF2\20ep\op\20240531_165251_NiyGB.png

0: 480x640 1 rectangle, 1 drop, 600.2ms
Speed: 2.1ms preprocess, 600.2ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)
Processed 20240531_165254_CACT9.png:
  Calculated width: 3.11727 mm
  Calculated D: 1.96273 mm
  Output saved to: C:\Users\baner\Desktop\BTP\btpf2\BTPF2-20241117T165856Z-001\BTPF2\20ep\op\20240531_165254_CACT9.png

0: 480x640 1 rectangle, 1 drop, 596.0ms
Speed: 2.6ms preprocess, 596.0ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)
Processed 20240531_170353_XfkEx.png:
  Calculated width: 2.64583 mm
  Calculated D: 1.61396 mm
  Output saved to: C:\Users\baner\Desktop\BTP\btpf2\BTPF2-